In [ ]:
import sctoolbox
from sctoolbox.utils import bgcolor

# 01 - Assembling or loading anndata object
<hr style="border:2px solid black"> </hr>

## 1 - Description
This notebook is dedicated to loading or creating an anndata object suitable for the subsequent analysis pipeline and optionally adding quality control-related metrics. The anndata object is prepared and finally stored as a `.h5ad` file. Based on the available data files there are multiple options to create the anndata object. To satisfy all and especially ATAC-related functionalities indexes are prepared to hold barcodes and feature coordinates.

### 1.1 Sources to create the anndata object:
1. **`.h5ad` file:**  
   Choose this option if you have a `.h5ad` file. The file could be provided by a preprocessing pipeline, a public dataset or a preceding analysis.

2. **Convert from R object:**  
   This option should be used if the data was processed using R. This can either be a `.rds` or `.robj` file.
   
3. **.mtx, barcode.tsv, [regions.tsv]**  
    Choose this option if you have the expression matrix in `.mtx` format, a file containing the barcodes (`*_barcodes.tsv`) and an optional file containing the regions (`*_regions.tsv`). Use this option for cases with the aforementioned three files available e.g. from a public dataset.

### 2.2 ATAC specific quality control metrics
1. **Nucleosome signal score:**  
   This ATAC specific quality control metric scores the fragment length distribution pattern of individual cells. This step utilizes PEAKQC for the score calculation. Choose if this score should be calculated and provide a file containing the fragments of the features. As input a bamfile with the raw reads or a bedfile containing fragments are suitable.

2. **Overlap with defined regions:**  
   Check for overlaps of the fragments with regions defined in a `.gtf` file. This could be promoters or transcription start sites. Choose if the overlap should be performed and provide the file holding the desired regions.


___________

## 2 - Setup

In [ ]:
# sctoolbox modules 
import sctoolbox.calc_overlap_pct as overlap
import peakqc.fld_scoring as fld
from sctoolbox.qc_filter import *
from sctoolbox.atac_utils import *
import sctoolbox.utils as utils
import sctoolbox.file_converter as converter

utils.settings_from_config("config.yaml", key="01")

___

## 3 - Read in data
<hr style="border:2px solid black"> </hr>

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
%bgcolor PowderBlue

# Choose one option

# For option 1: The path to an existing .h5ad file
path_h5ad = "test_data/scatac_pbmc.h5ad"

# For option 2: This is the path to the Seurat (.rds, .robj) file
path_rds = ""

# For option 3: Directory containing .mtx, barcodes.tsv and optionally regions.tsv
path_mtx = ""

----------------

In [ ]:
if sum(map(lambda x: x != "", [path_h5ad, path_rds])) != 1:
    del path_h5ad, path_mtx, path_rds
    raise ValueError("Please set only one of the above variables. Adjust the cell above and re-run.")

### 3.1 - Option 1: Read from h5ad

In [ ]:
if path_h5ad:
    adata = utils.load_h5ad(path_h5ad)

___

### 3.2 - Option 2: Convert from Seurat to anndata object

In [ ]:
# Converting from Seurat to anndata object
if path_rds:
    adata = converter.convertToAdata(file=path_rds)

____

### 3.3 - Option 3: .mtx, barcode.tsv, [regions.tsv]

In [ ]:
%bgcolor PowderBlue

# adjust in case of different naming schemes for any of the input files

mtx = '*matrix.mtx*'  # pattern for the file that contains counts
barcodes = '*barcodes.tsv*'  # pattern for the file that contains barcode information
variables = '*regions.tsv*'  # pattern for the optional file that contains variable information

In [ ]:
if path_mtx:
    adata = assembler.from_mtx(path_mtx, mtx=mtx, barcodes=barcodes, variables=variables, var_error=False)

____

## 4 - Prepare anndata
<hr style="border:2px solid black"> </hr>
Rename or remove `.obs` and `.var` columns as needed and format their indices. After this step the index of `.var` holds the feature coordinates and `.obs` the cell barcodes.

In [ ]:
import pandas as pd

with pd.option_context('display.max_rows', 5,'display.max_columns', None):
    display(adata.obs)
    display(adata.var)

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
%bgcolor PowderBlue

## 1. Modify existing columns

# .obs column names that should be deleted
drop_obs = []

# .obs column names that should be changed. E.g. "old_name": "New Name"
rename_obs = {}

# .var column names that should be deleted
drop_var = []

# .var column names that should be changed. E.g. "old_name": "New Name"
rename_var = {}


## 2. ATAC specific anndata properties

# columns where peak location data is stored (['chr', 'start', 'end'])
coordinate_cols = ['chr', 'start', 'end'] # (list:str) 

# should the adata.var index be formatted, that it matches chr:start-stop
set_index = True       # (boolean)

# should the .var index be generated from a certain column. Otherwise this is None 
index_from = None    # (str)

_________

### 4.1 - Rename and delete columns 

In [ ]:
# change obs
obs = adata.obs.copy()

obs.drop(columns=drop_obs, inplace=True)
obs.rename(columns=rename_obs, errors='raise', inplace=True)

# change var
var = adata.var.copy()

var.drop(columns=drop_var, inplace=True)
var.rename(columns=rename_var, errors='raise', inplace=True)

# apply changes to adata
adata.obs = obs
adata.var = var

___

### 4.2 - Format anndata indices

In [ ]:
adata = utils.prepare_atac_anndata(adata,
                   coordinate_cols=coordinate_cols,
                   set_index=set_index,
                   index_from=index_from)

________

## 5 - Add ATAC specific metrices
<hr style="border:2px solid black"> </hr>
Add ATAC specific QC-metrics to the `.obs` table.

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
%bgcolor PowderBlue

## 1. Source of fragments

# Either provide a bamfile or a bedfile containing fragments
fragments_file = 'test_data/scatac_pbmc_fragments.bed'

# If a bamfile was provided, which column contains the barcode information
barcode_tag = 'CB'

## 2. Choose actions to be done

# 2.1 calculate fragment length distribution score
calculate_fld_score = True

# 2.2 calculate overlap between fragments and regions
calculate_overlap = True

# Additional settings for the overlap
region_name = 'promoters'
regions_file = 'test_data/homo_sapiens.104.promoters2000.gtf'

# Number of threads available
threads = 8

---------

### 5.1 - Check barcode tag 
If a bamfile is provided this checks if the barcodes are available in the anndata object

In [ ]:
use_bam = fragments_file.endswith("bam")
if use_bam:
    check_barcode_tag(adata, fragments_file, barcode_tag)

____

### 5.2 - Score fragment length distributions

In [ ]:
if calculate_fld_score:

    fld.add_fld_metrics(adata=adata,
                        fragments=fragments_file,
                        barcode_col=None,
                        barcode_tag=barcode_tag,
                        chunk_size_bam=1000000,
                        regions=None,
                        peaks_thr=10,
                        wavelength=150,
                        sigma=0.4,
                        plot=False,
                        save_density=None,
                        save_overview=None,
                        sample=0)

    adata.obs

_____

### 5.3 - Calculate an overlap

In [ ]:
if calculate_overlap:
    
    if use_bam:
        fc_fragments_in_regions(adata=adata,
                                regions_file=regions_file,
                                bam_file=fragments_file,
                                cb_col=None,
                                cb_tag=barcode_tag,
                                regions_name=region_name,
                                threads=threads,
                                temp_dir=None)
        
    else:
        fc_fragments_in_regions(adata=adata,
                                regions_file=regions_file,
                                fragments_file=fragments_file,
                                cb_col=None,
                                regions_name=region_name,
                                threads=threads,
                                temp_dir=None)
        
    adata.obs

In [ ]:
adata.obs

________

## 6 - Saving the anndata object

In [ ]:
# Overview of loaded adata
display(adata)

In [ ]:
# Saving the data
adata_output = "anndata_1.h5ad"
utils.save_h5ad(adata, adata_output)

In [ ]:
sctoolbox.settings.close_logfile()